In [2]:
pip install openai


  Using cached openai-1.90.0-py3-none-any.whl.metadata (26 kB)
Using cached openai-1.90.0-py3-none-any.whl (734 kB)


In [16]:
# this is an example of how we can use a LLM with vision capabilities to scan through selected webcam footage for incidents related to spillage during waste collection

import os
import csv
import re
from openai import OpenAI
from PIL import Image
from io import BytesIO
import requests

# ✅ Set API key
os.environ["OPENAI_API_KEY"] ="MYKEY"
client = OpenAI()

# ✅ Image URLs (in real life, we'd have processed 1000s of clips from driver footage; they'd be compressed down and in a folder)
image_urls = [
    "https://thumbs.dreamstime.com/b/garbage-truck-accident-1807778.jpg?w=576",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEJUtZWg5zh1-hzdpg7OSXXVFXujEcAlAqog&s",
    "https://www.north-herts.gov.uk/sites/default/files/styles/3_2_landscape_795x530/public/2022-09/waste-purple-collection1500x1000.jpg?itok=Qr8vHi85",
    "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4y7j6krEVagxqfbd3bB6nSl9GB48fkBT3_w&s",
    "https://www.theargus.co.uk/resources/images/19313139/?type=responsive-gallery-fullscreen",
]


# ✅ Create CSV to record the results (later, we can use the images to train our models)
with open("image_descriptions.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["image_url", "description", "severity_score"])

    for img_url in image_urls:
        print(f"\n=== Processing: {img_url}")

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are a precise waste event scorer. Always follow instructions exactly."
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": (
                                # note on the prompt: I originally had a more generic prompt which gave severity score =7 for all images by being more descriptive and forcing chatgpt to forget previous images in this session, I got better results 
                                "You are a waste management expert. I want you to describe any visible spillage (if existing) "
                                "and give a numeric severity score from 1 (almost no mess) to 10 (extreme blockage).\n\n"
                                "Calibration examples:\n"
                                "Example 1:\n"
                                "Description: A single small piece of trash on the curb.\n"
                                "Severity: 1\n\n"
                                "Example 2:\n"
                                "Description: Large pile of garbage bags ripped open, trash scattered, blocking traffic.\n"
                                "Severity: 10\n\n"
                                "Now, for THIS IMAGE ONLY, reply ONLY in JSON:\n"
                                "{ \"description\": \"<your description>\", \"severity\": <1-10> }\n"
                                "NO extra text. NO code block."
                            )
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": img_url}
                        }
                    ]
                }
            ]
        )

        raw = response.choices[0].message.content.strip()
        print(f"Raw GPT output:\n{raw}\n")

        # ✅ Extract JSON
        json_match = re.search(r"\{[\s\S]*\}", raw)
        if json_match:
            json_str = json_match.group(0)
            try:
                data = json.loads(json_str)
                description = data.get("description", "")
                severity = data.get("severity", "")
            except Exception as e:
                print(f"❌ JSON parsing failed: {json_str}")
                description = ""
                severity = ""
        else:
            print(f"❌ No JSON found in output: {raw}")
            description = ""
            severity = ""

        writer.writerow([img_url, description, severity])

print("\n✅ CSV created: image_descriptions.csv")


=== Processing: https://thumbs.dreamstime.com/b/garbage-truck-accident-1807778.jpg?w=576
Raw GPT output:
{"description":"Garbage truck with back door open, large amount of trash including bags and loose waste spilled onto street, minor liquid leakage, not obstructing traffic.","severity":7}


=== Processing: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTEJUtZWg5zh1-hzdpg7OSXXVFXujEcAlAqog&s
Raw GPT output:
{"description": "A few pieces of paper have fallen next to trash bins, no significant spillage.", "severity": 2}


=== Processing: https://www.north-herts.gov.uk/sites/default/files/styles/3_2_landscape_795x530/public/2022-09/waste-purple-collection1500x1000.jpg?itok=Qr8vHi85
Raw GPT output:
{ "description": "No visible spillage; garbage is contained within a bin being loaded into a truck.", "severity": 1 }


=== Processing: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT4y7j6krEVagxqfbd3bB6nSl9GB48fkBT3_w&s
Raw GPT output:
{ "description": "Garbage truck with trash